In [53]:
%run data_plot.ipynb

In [54]:
dga_data = {'best_path': {},
            'possible_paths': {}}
solutions = [{}]

In [55]:
def swarm_init(origin, destination, swarm_pop):
    routes = []
    node_indices = [loc for loc in x_y_data.keys() if loc != origin and loc != destination]
    # generate population of length defined as a parameter
    for i in range(swarm_pop):
        # length of path (chromosome) is not fixed so generate a random length of path for each iteration
        path_length = randint(1, len(node_indices))
        # choose indices to be used as genes
        chosen_indices = choice(np.array(node_indices), path_length, replace=False)
        # prepend and append origin and destination respectively to the generated path
        route = np.append(origin, chosen_indices)
        route = np.append(route, destination)
        # add route to list of routes (the population)
        routes.append(route)

    return routes

In [56]:
swarm_idx = swarm_init(10, 151, 20)
swarm_idx

[array([ 10, 118,  60, 139, 117,  98, 112, 120, 150,  81, 133,  95,  64,
        130, 135, 126, 109,   1,  16, 110, 102,  61,  37,  41, 111, 137,
         42, 129, 152,  90,  65,  83,  30,  49, 116,  67,  11,  80,   4,
         47, 144,  62, 104, 100,  96,  58,  36, 145,  55,  26,  38,  28,
         39, 128,  24, 148,  56, 147,  89,   2,  57,  71, 123,  75,  35,
         12,  69, 132,  18, 140,  85,  91, 143,  48, 141,  50,  63,  93,
          3,  74,  86,  13,  77,  25, 121,   5,  29,  27, 119,  53, 149,
         68, 122,  54, 124,  92,  70,  82, 146,  78, 142,  99, 103,  51,
         94,  66,  97,  44, 134, 138, 108, 131,   8,  43, 101,  15,  32,
        113,  14, 125,  59, 106, 136,  46,  34,  72,   7,  52,  23, 105,
         21, 151]),
 array([ 10,  14,  39,  64,  66, 122, 120,  52,  78, 121, 138,  21,  53,
         51,  43, 136,  32, 118,  24,  58,  89,  98, 144,   2, 117,  50,
        148, 140,  45, 104,  34, 115, 128,  41, 105,  47, 100,  25, 102,
         42,  86,  81,  11,  69

In [57]:
def calculate_route_cost(swarm_idx):
    global dga_data, solutions
    route_costs = []
    for particle in swarm_idx:
        path_bandwidths = []
        path_latency = 0
        for var in range(len(particle) - 1):
            # get path distance
            path_distance = calculate_distance(x_y_data[particle[var]], x_y_data[particle[var + 1]])
            # get bandwidth between node and its next node and append it to path bandwidths var
            path_bandwidth = convert_distance_bandwidth(path_distance)
            path_latency += node_to_node_latency
            path_bandwidths.append(path_bandwidth)
        # computed end-to-end bandwidth rate
        end_to_end_bandwidth = min(path_bandwidths)
        # step latency down to 0 if bandwidth is 0
        if end_to_end_bandwidth == 0:
            absolute_path_cost = 0
        else:
            absolute_path_cost = round(absolute_cost_function(end_to_end_bandwidth, path_latency), 3)
            if len(dga_data['best_path']) == 0:
                solutions[0] = {"source node": f"Node-{route[0]}"}
                routing_path = ''
                for idx, sensor in enumerate(route[1:]):
                    routing_path += f"(Node-{sensor}, {path_bandwidths[idx]} Mbps),"
                solutions[0]["routing path"] = routing_path
                solutions[0]["end-to-end transmission rate"] = f"{end_to_end_bandwidth} Mbps"
                solutions[0]["end-to-end latency"] = f"{path_latency} ms"
                solutions[0]["absolute path cost"] = f"{absolute_path_cost}"
                dga_data['best_path'][str(list(route))] = {'end_to_end_bandwidth': end_to_end_bandwidth,
                                                     'path_latency': path_latency,
                                                     'absolute_cost': absolute_path_cost
                                                    }
            else:
                solutions[0] = {"source node": f"Node-{route[0]}"}
                routing_path = ''
                for idx, sensor in enumerate(route[1:]):
                    routing_path += f"(Node-{sensor}, {path_bandwidths[idx]} Mbps),"
                solutions[0]["routing path"] = routing_path
                solutions[0]["end-to-end transmission rate"] = f"{end_to_end_bandwidth} Mbps"
                solutions[0]["end-to-end latency"] = f"{path_latency} ms"
                solutions[0]["absolute path cost"] = f"{absolute_path_cost}"
                best_path = dga_data['best_path'][list(dga_data['best_path'].keys())[0]]
                if absolute_path_cost > best_path['absolute_cost']:
                    dga_data['possible_paths'][list(dga_data['best_path'].keys())[0]] = {
                                                     'end_to_end_bandwidth': best_path['end_to_end_bandwidth'],
                                                     'path_latency': best_path['path_latency'],
                                                     'absolute_cost': best_path['absolute_cost']
                                                    }
                    dga_data['best_path'].pop(list(dga_data['best_path'].keys())[0])
                    dga_data['best_path'][str(list(route))] = {
                                                     'end_to_end_bandwidth': end_to_end_bandwidth,
                                                     'path_latency': path_latency,
                                                     'absolute_cost': absolute_path_cost
                                                    }
                else:
                    dga_data['possible_paths'][str(list(route))] = {
                                                     'end_to_end_bandwidth': end_to_end_bandwidth,
                                                     'path_latency': path_latency,
                                                     'absolute_cost': absolute_path_cost
                                                    }
        # add end-to-end bandwidth to list containing all cost data
        route_costs.append(absolute_path_cost)

    return route_costs

In [58]:
costs = calculate_route_cost(swarm_idx)